# Métodos de classificação para mapas temáticos

### Importamos as bibiotecas

In [ ]:
import json
import geopandas as gpd
import branca.colormap as cm
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import zipfile
import folium
from jenkspy import jenks_breaks
from folium.plugins import MarkerCluster, Fullscreen
from tqdm import tqdm  # Para barra de progresso (opcional)
import os
import re
import math


### Visualizamos o shapefile

In [ ]:
# Configurações
COD_MUNICIPIO = '3136702'
PATH_ZIP = "MG_setores_CD2022.zip"
PATH_EXTRACTED = "MG_setores_CD2022"
PATH_OUTPUT = "JF_setores"
MAX_FEATURES = 5000  # Limite máximo de features para carregar (ajuste conforme sua RAM)

# Extração otimizada
if not os.path.exists(PATH_EXTRACTED):
    print("Descompactando arquivo ZIP...")
    with zipfile.ZipFile(PATH_ZIP, 'r') as zip_ref:
        # Extrai apenas os arquivos essenciais
        essential_files = [f for f in zip_ref.namelist() if f.endswith(('.shp', '.shx', '.dbf'))]
        for file in tqdm(essential_files, desc="Extraindo"):
            zip_ref.extract(file, PATH_EXTRACTED)

# Carregamento otimizado
def load_and_filter():
    # Encontra o arquivo .shp
    shp_file = next((f for f in os.listdir(PATH_EXTRACTED) if f.endswith('.shp')), None)
    if not shp_file:
        raise FileNotFoundError("Arquivo .shp não encontrado na pasta extraída")
    
    shp_path = os.path.join(PATH_EXTRACTED, shp_file)
    
    # Carrega apenas as colunas necessárias
    print("Carregando e filtrando dados...")
    gdf = gpd.read_file(
        shp_path,
        rows=MAX_FEATURES,  # Limita a quantidade de registros
        columns=['CD_MUN', 'CD_SETOR', 'AREA_KM2', 'geometry'],  # Apenas colunas essenciais
        where=f"CD_MUN = '{COD_MUNICIPIO}'"  # Filtra diretamente na leitura
    )
    
    # Simplifica geometrias
    gdf['geometry'] = gdf['geometry'].simplify(tolerance=0.0005)
    return gdf

# Execução principal
if os.path.exists(f"{PATH_OUTPUT}.geojson"):
    print("Carregando dados processados...")
    gdf_jf = gpd.read_file(f"{PATH_OUTPUT}.geojson")
else:
    gdf_jf = load_and_filter()
    gdf_jf.to_file(f"{PATH_OUTPUT}.geojson", driver='GeoJSON')
    print(f"Dados salvos em {PATH_OUTPUT}.geojson")

# Visualização segura
print(f"\nTotal de setores carregados: {len(gdf_jf)}")
if not gdf_jf.empty:
    # Criar a figura
    fig, ax = plt.subplots(figsize=(10, 8))
    
    # Plotar o mapa
    gdf_jf.plot(ax=ax, column='CD_SETOR', legend=False)
    ax.set_title(f"Setores Censitários de Juiz de Fora")
    
    # Salvar como PNG
    #plt.savefig('mapa_setores_jf.png', dpi=300, bbox_inches='tight')
    #print("Mapa salvo como 'mapa_setores.png'")
    
    # Mostrar o mapa
    plt.show()
#else:
    print("Nenhum dado foi carregado. Verifique o código do município.")

**Verificamos o data frame**

In [ ]:
gdf_jf.info()

**Carregamos e processamos os dados**

In [ ]:
with zipfile.ZipFile('3136702_JUIZ_DE_FORA.zip') as z:
    with z.open('3136702_JUIZ_DE_FORA.csv') as f:
        chunks = pd.read_csv(f, 
                           sep=';',
                           chunksize=1000,
                           usecols=['COD_ESPECIE', 'COD_SETOR', 'LATITUDE', 'LONGITUDE'])
        
        # Carregar todos os dados das categorias 3 e 6
        filtered_data = pd.concat([
            chunk[chunk['COD_ESPECIE'].isin([6,6])] 
            for chunk in chunks
        ])


**Agrupamos por setor censitário**

In [ ]:
contagem_por_setor = filtered_data.groupby('COD_SETOR').size().reset_index(name='CONTAGEM')

In [ ]:
print(contagem_por_setor.head())

**Carregar geojson com os polígonos dos setores**

In [ ]:
try:
    with open('JF_setores.geojson') as f:
        geojson_data = json.load(f)
except FileNotFoundError:
    print("Arquivo GeoJSON não encontrado.")
    geojson_data = {
        "type": "FeatureCollection",
        "features": []
    }

**Preparamos os dados para o mapa coroplético**<br>
Criamos dicionário de contagem removendo o caracter final para corresponder ao GeoJSON


In [ ]:
contagem_dict = contagem_por_setor.copy()
contagem_dict['COD_SETOR'] = contagem_dict['COD_SETOR'].str.extract('(\\d+)')[0]
contagem_dict = contagem_dict.set_index('COD_SETOR')['CONTAGEM'].to_dict()

**Realizamos diagnóstico de correspondência das chaves**

In [ ]:
# Amostras das chaves
csv_keys = list(contagem_dict.keys())[:10]
geojson_keys = [f['properties']['CD_SETOR'] for f in geojson_data['features'][:10]]

print("Chaves do CSV:", csv_keys)
print("Chaves do GeoJSON:", geojson_keys)

# Verificar sobreposição
common_keys = set(contagem_dict.keys()) & set(f['properties']['CD_SETOR'] for f in geojson_data['features'])
print(f"\nChaves em comum: {len(common_keys)} de {len(geojson_data['features'])}")

**Adicionamos dados ao GeoJSON**

In [ ]:
for feature in geojson_data['features']:
    cod_setor = feature['properties']['CD_SETOR']
    feature['properties']['CONTAGEM'] = contagem_dict.get(cod_setor, float('nan'))

**Verificamos os dados finais**

In [ ]:
print("\nVerificação final da correspondência:")
for feature in geojson_data['features'][:5]:  # Mostrar apenas os 5 primeiros para exemplo
    cod = feature['properties']['CD_SETOR']
    print(f"Setor: {cod}, Contagem: {feature['properties']['CONTAGEM']}")

**Função para inicializar o mapa**<br>

Para evitar que novas camadas sejam sobrepostas às anterioes, criamos uma função para reinicializar o objeto *folium.Map* a cada chamada.
Calculamos a média das coordenadas para encontrar o centro geográfico ou *baricentro* do conjunto de pontos.

In [ ]:
def initMap():
    tiles = 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png'
    attr = '&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors'

    center_lat = filtered_data['LATITUDE'].mean()
    center_lon = filtered_data['LONGITUDE'].mean()

    map = folium.Map(location=[center_lat, center_lon],
                zoom_start = 10,
                tiles = tiles,
                attr = attr)
    return map

**Definimos a escala de cores**<br>

O código a abaixo cria um mapeamento dinâmico onde valores maiores recebem cores mais escuras/intensas, facilitando visualizações como mapas de calor ou gráficos temáticos. Os dados discretos são representados por uma escala de cores contínua.

In [ ]:
max_contagem = contagem_por_setor['CONTAGEM'].max()
colormap = cm.linear.YlOrRd_09.scale(0, max_contagem)

**Adicionamos a camada coroplética**

Para gerar o mapa coroplético podemos usar a classe *folium.Choropleth* ou a classe *folium.GeoJson*. A primeira opção é mais conveniente para casos simples, enquanto a segunda oferece mais flexibilidade para personalizações avançadas.

Demonstramos a seguir o uso da classe *folium.Choroplet*, para fins didáticos.<br>
Depois, utilizaremos apenas a classe *folium.GeoJson*.

**Exemplo de uso da classe *folium.Choroplet***

In [ ]:
contagem = contagem_por_setor.copy()
contagem['COD_SETOR'] = contagem['COD_SETOR'].str.extract('(\\d+)')[0]
#contagem = contagem_dict.set_index('COD_SETOR')['CONTAGEM'].to_dict()

m = initMap()

folium.Choropleth(
    geo_data=geojson_data,
    name='Densidade de Estabelecimentos',
    data=contagem,
    columns=['COD_SETOR', 'CONTAGEM'],  # Coluna de junção e coluna de valores
    key_on='feature.properties.CD_SETOR',  # Caminho para a chave no GeoJSON
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    line_weight=0.5,
    line_color='white',
    legend_name='Número de Estabelecimentos',
    bins=6,  # Número de classes
    nan_fill_color = 'grey',
    reset=True
).add_to(m)



display(m)

**Adotamos a classe *folium.GeoJson***

In [ ]:
m = initMap()

folium.GeoJson(
    geojson_data,
    name='Densidade de Estabelecimentos Comerciais',
    style_function=lambda feature: {
        'fillColor': 'grey' if math.isnan(feature['properties']['CONTAGEM']) 
                            else colormap(feature['properties']['CONTAGEM']),
        'color': 'white',
        'weight': 0.5,
        'fillOpacity': 0.7,
        'line_opacity': 0.2
            },
).add_to(m)

display(m)

**Análise preliminar**

In [ ]:
contagem_por_setor.describe()

In [ ]:
contagem_por_setor.boxplot(vert=False)

**Normalização dos dados**

In [ ]:
# Adicionar dados ao GeoJSON e calcular densidade
densidades_validas = []  # Lista para coletar densidades válidas

for feature in geojson_data['features']:
    cod_setor = feature['properties']['CD_SETOR']
    contagem = contagem_dict.get(cod_setor, float('nan'))
    area_km2 = feature['properties'].get('AREA_KM2', 1)
    
    # Calcular densidade
    if not math.isnan(contagem) and area_km2 > 0:
        densidade = contagem / area_km2
        densidades_validas.append(densidade)  # Coletar para cálculo do percentil
    else:
        densidade = float('nan')
    
    feature['properties']['CONTAGEM'] = contagem
    feature['properties']['DENSIDADE'] = densidade

m = initMap()

# Escala de cores para densidade - USAR APENAS VALORES VÁLIDOS
if densidades_validas:
    percentil_95 = np.percentile(densidades_validas, 95)
    colormap_dens = cm.linear.YlOrRd_09.scale(0, percentil_95)
else:
    # Fallback caso não haja dados válidos
    colormap_dens = cm.linear.YlOrRd_09.scale(0, 1)

folium.GeoJson(
    geojson_data,
    name='Densidade de Estabelecimentos Comerciais',
    style_function=lambda feature: {
        'fillColor': 'grey' if math.isnan(feature['properties']['DENSIDADE']) 
                            else colormap_dens(feature['properties']['DENSIDADE']),
        'color': 'white',
        'weight': 0.5,
        'fillOpacity': 0.7,
        'line_opacity': 0.2
        # Removi 'nan_fill_color' pois já estamos tratando manualmente
    },
).add_to(m)

display(m)

**Métodos de classificação**


In [ ]:

def criar_mapa_dropdown_simples():
    valores = [v for v in contagem_dict.values() if not math.isnan(v)]  # Apenas valores válidos
    n_classes = 6

    # Calcular breaks apenas com valores válidos
    breaks_equal = np.linspace(min(valores), max(valores), n_classes + 1)
    breaks_quantiles = np.quantile(valores, np.linspace(0, 1, n_classes + 1))
    breaks_jenks = jenks_breaks(valores, n_classes=n_classes)

    # Mapa base
    m = initMap()

    # Criar FeatureGroups para cada método
    fg_equal = folium.FeatureGroup(name='Equal Interval', show=True)
    fg_quantiles = folium.FeatureGroup(name='Quantiles', show=False)
    fg_jenks = folium.FeatureGroup(name='Jenks', show=False)

    # Adicionar camadas
    def adicionar_camada(feature_group, breaks, colormap):
        def style_func(feature):
            cod_setor = feature['properties']['CD_SETOR']
            valor = contagem_dict.get(cod_setor, float('nan'))  # MODIFICADO: usa NaN
            
            # Verificar se é NaN (sem dados)
            if math.isnan(valor):
                return {
                    'fillColor': 'grey',
                    'color': 'white',
                    'weight': 0.5,
                    'fillOpacity': 0.3,
                    'line_opacity': 0.2
                }
            
            # Para valores válidos, encontrar a classe correspondente
            for i in range(len(breaks)-1):
                if breaks[i] <= valor < breaks[i+1]:
                    return {
                        'fillColor': colormap(breaks[i]),
                        'color': 'white',
                        'weight': 0.5,
                        'fillOpacity': 0.7,
                        'line_opacity': 0.2
                    }
            
            # Se valor for maior que o último break (caso do máximo)
            return {
                'fillColor': colormap(breaks[-2]),
                'color': 'white',
                'weight': 0.5,
                'fillOpacity': 0.7,
                'line_opacity': 0.2
            }

        folium.GeoJson(geojson_data, style_function=style_func).add_to(feature_group)
        feature_group.add_to(m)


    # Equal Interval (visível por padrão)
    cmap_equal = cm.linear.YlOrRd_09.to_step(n=n_classes, index=breaks_equal)
    adicionar_camada(fg_equal, breaks_equal, cmap_equal)

    # Quantiles
    cmap_quantiles = cm.linear.YlOrRd_09.to_step(n=n_classes, index=breaks_quantiles)
    adicionar_camada(fg_quantiles, breaks_quantiles, cmap_quantiles)

    # Jenks
    cmap_jenks = cm.linear.YlOrRd_09.to_step(n=n_classes, index=breaks_jenks)
    adicionar_camada(fg_jenks, breaks_jenks, cmap_jenks)

    # Adicionar plugin de tela cheia
    fullscreen_plugin = Fullscreen(
        position='bottomleft',
        title='Expandir tela',
        title_cancel='Sair da tela cheia',
        force_separate_button=True
    ).add_to(m)

    # Controle de camadas
    folium.LayerControl(collapsed=False).add_to(m)

    # Salvar mapa como arquivo HTML
    m.save('mapa_clas.html', close_file=True)
    
    return m


# Executar versão corrigida
mapa_clas = criar_mapa_dropdown_simples() 

display(mapa_clas)

**Escala logarítmica**

Uma outra maneira de padronizar os dados, caso sejam assimétricos, consiste em convertê-los para a escala logarítmica. Nesse caso, é desnecessária a adoção de algum método de classificação.

In [ ]:
# Criar escala logarítmica
contagem_por_setor['MapScale'] = np.log10(contagem_por_setor['CONTAGEM'])
contagem_dict = contagem_por_setor.copy()
contagem_dict['COD_SETOR'] = contagem_dict['COD_SETOR'].str.extract('(\\d+)')[0]

contagem_dict = contagem_dict.set_index('COD_SETOR')['MapScale'].to_dict()

# Adicionar dados ao GeoJSON
for feature in geojson_data['features']:
    cod_setor = feature['properties']['CD_SETOR']
    feature['properties']['MapScale'] = contagem_dict.get(cod_setor, float('nan'))


In [ ]:
contagem_por_setor.boxplot(column=['MapScale'], vert=False)

In [ ]:
# Escala de cores
max_contagem = contagem_por_setor['MapScale'].max()
colormap_log = cm.linear.YlOrRd_09.scale(0, max_contagem)

In [ ]:
m = initMap()

folium.GeoJson(
    geojson_data,
    style_function=lambda feature: {
        'fillColor': 'grey' if math.isnan(feature['properties']['MapScale']) 
                            else colormap_log(feature['properties']['MapScale']),
        'color': 'white',
        'weight': 0.5,
        'fillOpacity': 0.7,
        'line_opacity': 0.2,
        'nan_fill_color': 'grey'
    }
).add_to(m)

display(m)

In [ ]:
# Calcular valores para a legenda baseados nos valores originais
max_contagem_log = contagem_por_setor['MapScale'].max()
min_contagem_original = contagem_por_setor['CONTAGEM'].min()
max_contagem_original = contagem_por_setor['CONTAGEM'].max()

# Criar colormap com valores originais
colormap = cm.linear.YlOrRd_09.scale(min_contagem_original, max_contagem_original)

# Criar ticks personalizados para a legenda com valores originais
log_ticks = np.linspace(0, max_contagem_log, 6)
custom_ticks = [round(10**x) for x in log_ticks if x >= 0]
if min_contagem_original < custom_ticks[0]:
    custom_ticks.insert(0, min_contagem_original)
if max_contagem_original > custom_ticks[-1]:
    custom_ticks.append(max_contagem_original)

# Configurar legenda personalizada
colormap.caption = 'Quantidade de estabelecimentos comerciais por Setor' 
                        
colormap.add_to(m)

# Adicionar plugin de tela cheia
fullscreen_plugin = Fullscreen(
    position='bottomleft',
    title='Expandir tela',
    title_cancel='Sair da tela cheia',
    force_separate_button=True
).add_to(m)

# Exibir o mapa
display(m)